<a href="https://colab.research.google.com/github/jhodgens17/audio_classification/blob/main/Third_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#Import packages
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
targets = ['down', 'go', 'left', 'no', 'off',
           'on', 'right', 'stop', 'up', 'yes']

In [9]:
all_files = {}
for target in targets:
  path = '/content/drive/MyDrive/train/audio/'+target
  target_list = os.listdir(path)
  all_files[target] = target_list

In [10]:
len(all_files)

10

In [12]:
for target in all_files:
  print(len(all_files[target]))

2359
2372
2353
2375
2357
2367
2367
2380
2375
2377


In [13]:
all_files['up'][0]

'a4baac4e_nohash_0.wav'

In [14]:
all_paths = {}
for target in all_files:
  paths = []
  for file in all_files[target]:
    path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
    paths.append(path)
  all_paths[target] = paths

In [16]:
len(all_paths)

10

In [17]:
for target in all_paths:
  print(len(all_paths[target]))

2359
2372
2353
2375
2357
2367
2367
2380
2375
2377


In [18]:
all_paths['stop'][0]

'/content/drive/MyDrive/train/audio/stop/15c563d7_nohash_1.wav'

In [19]:
#Defining function to repeat process to get waveform for each clip
def get_waveform(path):
  audio_binary = tf.io.read_file(path)
  audio, sample_rate = tf.audio.decode_wav(audio_binary)
  waveform = tf.squeeze(audio, axis=-1)
  return waveform

In [20]:
#Defining a function that makes all audio clips same length to get spectrogram
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [22]:
y = []
all_paths = []
for target in all_files:
  for file in all_files[target]:
    path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
    all_paths.append(path)
    y.append(target)

In [25]:
#Getting all of the files
files_ds = tf.data.Dataset.from_tensor_slices(all_paths)

In [26]:
#Getting the waveform for each file
waveform_ds = files_ds.map(get_waveform)

In [27]:
#Getting the spectrogram for each file
spectrogram_ds = waveform_ds.map(get_spectrogram)

In [28]:
X = []
for element in spectrogram_ds:
  X.append(element.numpy())

In [29]:
X = np.array(X)
y = np.array(y)

In [40]:
y

array(['down', 'down', 'down', ..., 'yes', 'yes', 'yes'], dtype='<U5')

In [41]:
import pandas as pd

In [43]:
y = pd.get_dummies(y)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [54]:
model = Sequential()

In [46]:
for spectrogram in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)

Input shape: (124, 129)


In [47]:
X.shape

(23682, 124, 129)

In [48]:
num_labels = 10

In [55]:
model.add(Input(shape=input_shape))
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_labels))

In [56]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
278/278 [==============================] - 2s 5ms/step - loss: 0.7819 - accuracy: 0.1247
Epoch 2/50
278/278 [==============================] - 1s 5ms/step - loss: 0.4679 - accuracy: 0.1709
Epoch 3/50
278/278 [==============================] - 1s 5ms/step - loss: 0.4557 - accuracy: 0.1990
Epoch 4/50
278/278 [==============================] - 1s 5ms/step - loss: 0.4559 - accuracy: 0.2183
Epoch 5/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4584 - accuracy: 0.2218
Epoch 6/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4526 - accuracy: 0.2278
Epoch 7/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4515 - accuracy: 0.2298
Epoch 8/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4487 - accuracy: 0.2320
Epoch 9/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4532 - accuracy: 0.2374
Epoch 10/50
278/278 [==============================] - 1s 4ms/step - loss: 0.4431 - accuracy: 0.2297

In [73]:
y_pred = np.argmax(model.predict(X_test), axis = 1)

In [79]:
y_test_arr = y_test.idxmax(axis = 1)

In [80]:
y_pred

array([6, 8, 6, ..., 4, 6, 7])

In [81]:
y_test.columns

Index(['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes'], dtype='object')

In [84]:
tar_dict = {0:'down', 1:'go', 2:'left', 3:'no', 4:'off',
            5:'on', 6:'right', 7:'stop', 8:'up', 9:'yes'}

In [85]:
y_hat = []
for value in y_pred:
  y_hat.append(tar_dict[value])

In [88]:
y_hat_arr = np.array(y_hat)

In [93]:
accuracy_score(y_test_arr, y_hat_arr)

0.2633001182232731